# Experimental data conversion and display

In [2]:
# Not all of these are needed here
import sys,os, glob
import numpy as np
#from tools import tex,checkdir,load,save,fill_between
#from tools import tex,plot_band,fill_between
import pylab as py
from  matplotlib import rc
import matplotlib.gridspec as gridspec
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'  
rc('font',**{'family':'sans-serif','sans-serif':['Times-Roman']})
rc('text',usetex=True)
from scipy.integrate import quad
import lhapdf
import pandas as pd
from  scipy.interpolate import  interp1d
from scipy.interpolate import spline
import scipy.interpolate
from scipy.signal import savgol_filter
from scipy.stats import gaussian_kde,norm,ks_2samp
from scipy.optimize import minimize,root

## HERA I+II combination

In [38]:
## transforms the HERA I+II combination into CJ format
def HERA_12_to_CJ(infile,outfile,comment):
  F=open(infile)
  L=F.readlines()
  F.close()
    
  ### exp. data
  L=[l.strip().split() for l in L if l.strip()!='']
  #L=L[0:4]
  
  mod=0    # removes unwanted columns (tot_nonproc)
  for col in range(len(L[0])):
    if 'tot' in L[0][col+mod]:
      L = np.delete(L,col+mod,axis=1)
      mod-=1
  D = L[1:] ## actual data (removes first row of headers)  
  D = map(lambda myList:map(float,myList),D)   # transforms into a matrix of floats
  D = np.array(D)
  
  ### Swaps the Q2 and x  columns  (b/c CJ likes x, then Q2)
  D[:,[0, 1]] = D[:,[1, 0]]

  ### calculates total syst error = sum in quadrature of uncorr and all teh correlated errors
  ### and inserts them in 6th position 
  syst = []
  for d in D:
    s = (sum(map(lambda x:x**2,d[6:]))+d[5]**2)**0.5
    syst.append(s)
  syst = np.array(syst)
  D = np.insert(D,5,syst,axis=1)
  
  ### converts stat, syst, and uncorrelated syst from % of cross section to absolute value
  Sigma = D[:,3:4]  # cross sections
  D[:,4:7] = Sigma * D[:,4:7] / 100  # Each column in the D slice is multiplied element by element with Sigma
  
  # Manual headers for now  (may think of an automatic way of formatting the original ones later)
  H = '    x            Q2           y           sig_r         stat         syst         uncor         cor1        cor2         ...\n'
  
  #saves to outfile
  F=open(outfile,'w')
  F.write('# HERA I+II combined data for '+comment+'\n') 
  F.write('# http://arxiv.org/abs/1506.06042v2 and http://hepdata.cedar.ac.uk/view/ins1377206\n')
  F.write('# Uncorrelated errors as fraction of xsec; correlated errors in % of measured cross section\n')
  F.write('.000  169 : Normalization error, no. of correlated errors\n')
  F.write(H)
  np.savetxt(F,D,fmt='% .5e')   ### could be improved by using only '% .3f' for the correlated errors
  F.close()

  return H,D

In [39]:
root='HERA_I+II_combination/'
HERA_12_to_CJ(root+'HERA1+2_CCem.dat','H2_CC_em','electron charged current');
HERA_12_to_CJ(root+'HERA1+2_CCep.dat','H2_CC_ep','positron charged current');
HERA_12_to_CJ(root+'HERA1+2_NCem.dat','H2_NC_em','electron charged current');
HERA_12_to_CJ(root+'HERA1+2_NCep_920.dat','H2_NC_ep_1','positron neutral current E=920 GeV');
HERA_12_to_CJ(root+'HERA1+2_NCep_820.dat','H2_NC_ep_2','positron neutral current E=820 GeV');
HERA_12_to_CJ(root+'HERA1+2_NCep_575.dat','H2_NC_ep_3','positron neutral current E=575 GeV');
HERA_12_to_CJ(root+'HERA1+2_NCep_460.dat','H2_NC_ep_4','positron neutral current E=460 GeV');